<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:100px; margin: 20px;" />
</div>

## Welcome to UDF Creation
In this notebook we are going to create a UDF and store it inside the Exasol database for executing the predict function of our model.

#### Open Configuration 

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

#### Create UDF
The UDF we are creating below is using whats called "variadic script". Which means the number of input and output parameters are flexible. 
The magic setting for variadic scripts is writing "…" in the parameter list.
When we say CREATE OR REPLACE PYTHON3 SET SCRIPT XGBOOST(…), then this XGBOOST script can be called with
0, with 7, with 50, or with any number of parameters. 
    
We are using open_pyexasol_connection_with_lang_definitions() to access our cutomized SLC that we just made from the script_languages_container folder.

In [ ]:
from exasol.nb_connector.slc import ScriptLanguageContainer
slc = ScriptLanguageContainer(secrets=ai_lab_config, name="sample_slc_6")

In [ ]:
import textwrap
from exasol.nb_connector.language_container_activation import open_pyexasol_connection_with_lang_definitions
from exasol.nb_connector.connections import open_pyexasol_connection
from stopwatch import Stopwatch

stopwatch = Stopwatch()


# Create script to test the model
sql = textwrap.dedent(f"""\
CREATE OR REPLACE {slc.language_alias} SET SCRIPT
{{schema!q}}.XGBOOST(...)
EMITS(...) AS

import pickle
import pandas as pd


def run(ctx):
    # Load model from EXABucket
    with open(ctx[0], 'rb') as f:
        model = pickle.load(f)
        
    # Stream the data through the model to reduce the required main memory of the UDF.
    # This allows running the UDF on larger datasets.
    while True:
        # Read the input skipping the first column which holds the model path.
        X_pred = ctx.get_dataframe(num_rows=1000, start_col=1)
        if X_pred is None:
            break

        # Call the model to get the predictions. Omit the first column in the input
        # which holds the sample IDs.
        df_features = X_pred.drop(X_pred.columns[0], axis=1)
        y_pred = model.predict(df_features.values)

        # Combine predictions with the sample IDs.
        df_rowid = X_pred[X_pred.columns[0]].reset_index(drop=True)
        df_pred = pd.concat((df_rowid, pd.DataFrame(y_pred)), axis=1)

        # Output data
        ctx.emit(pd.DataFrame(y_pred))
/
""")

conn_lang = open_pyexasol_connection_with_lang_definitions(ai_lab_config, schema=ai_lab_config.db_schema, compression=True)
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn_lang.execute(query=sql, query_params={'schema': ai_lab_config.db_schema})
    rs= conn_lang.execute("""select execute_shell_command_py3('python3 -m pip list', true)""")
    for r in rs: 
        print(r[0])

print(f"Creating prediction script took: {stopwatch}")

We can now move to the [main](./2-main.ipynb) file to train and upload our model.